In [60]:
# Make division default to floating-point, saving confusion
from __future__ import division
from __future__ import print_function

# Allowed libraries
import numpy as np
import pandas as pd
import scipy as sp
import heapq as pq
import matplotlib as mp
import math
from itertools import product, combinations
from collections import OrderedDict as odict
from graphviz import Digraph
from tabulate import tabulate

In [129]:
#Learn the outcomespace

def learn_outcome_space(data):
    outcomeSpace=dict()
    for i in data.columns:
        outcomeSpace[i]=tuple(data[i].unique())
    return outcomeSpace
data = pd.read_csv('data.csv')
#first column is index in excel
del data[data.columns[0]]

#We are going to consider just if a room is empty or not (we don't care about the exact number of people)
room_columns=[]
for i in range(1,36):
    room_columns.append('r'+str(i))
#room_columns=tuple(room_columns)
for i in room_columns:
    data[i]=data[i].replace(to_replace=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],value=1)    
#    data[i].where(~(data.i!=0),other=1,inplace=True)

outcomeSpace = learn_outcome_space(data)

In [137]:
def printFactor(f):
    """
    argument 
    `f`, a factor to print on screen
    """
    # Create a empty list that we will fill in with the probability table entries
    table = list()
    
    # Iterate over all keys and probability values in the table
    for key, item in f['table'].items():
        # Convert the tuple to a list to be able to manipulate it
        k = list(key)
        # Append the probability value to the list with key values
        k.append(item)
        # Append an entire row to the table
        table.append(k)
    # dom is used as table header. We need it converted to list
    dom = list(f['dom'])
    # Append a 'Pr' to indicate the probabity column
    dom.append('Pr')
    print(tabulate(table,headers=dom,tablefmt='orgtbl'))

In [138]:
# Dependencies for transition probabilities
previous_G = {
    'r1' : ['r1','r2','r3'],
    'r2' : ['r1', 'r2','r4'],
    'r3' : ['r1', 'r3','r7'],
    'r4' : ['r2','r4','r8'],
    'r5' : ['r5','r6', 'r9'],
    'r6' : ['r5','r6'],
    'r7' : ['r3', 'r7'],
    'r8' : ['r4', 'r8','r9'],
    'r9' : ['r5', 'r8','r9', 'r13'],
    'r10': ['r10'],
    'r11': ['r11'],
    'r12': ['r12','r22', 'outside'],
    'r13': ['r9', 'r13','r24'],
    'r14': ['r14','r24'],
    'r15': ['r15'],
    'r16': ['r16'],
    'r17': ['r17'],
    'r18': ['r18'],
    'r19': ['r19'],
    'r20': ['r20'],
    'r21': ['r21'],
    'r22': ['r12', 'r22','r25'],
    'r23': ['r23','r24'],
    'r24': ['r13', 'r14', 'r23','r24'],
    'r25': ['r22', 'r25','r26'],
    'r26': ['r25', 'r26','r27'],
    'r27': ['r26', 'r27','r32'],
    'r28': ['r28'],
    'r29': ['r29','r30'],
    'r30': ['r29','r30'],
    'r31': ['r31','r32'],
    'r32': ['r27', 'r31','r32', 'r33'],
    'r33': ['r32','r33'],
    'r34': ['r34'],
    'r35': ['r35'],
    'c1' : ['r7', 'r25'],
    'c2' : ['r34'],
    'c3' : ['r5', 'r6', 'r10', 'r11', 'r15', 'r16', 'r17', 'r18', 'r19', 'r20', 'r21'],
    'c4' : ['r28', 'r29', 'r35'],
    'o1' : ['c3', 'c4'],
    'outside': ['r12']
}

In [149]:
def allEqualThisIndex(dict_of_arrays, **fixed_vars):
    """
    Helper function to create a boolean index vector into a tabular data structure,
    such that we return True only for rows of the table where, e.g.
    column_a=fixed_vars['column_a'] and column_b=fixed_vars['column_b'].
    
    This is a simple task, but it's not *quite* obvious
    for various obscure technical reasons.
    
    It is perhaps best explained by an example.
    
    >>> all_equal_this_index(
    ...    {'X': [1, 1, 0], Y: [1, 0, 1]},
    ...    X=1,
    ...    Y=1
    ... )
    [True, False, False]
    """
    # base index is a boolean vector, everywhere true
    first_array = dict_of_arrays[list(dict_of_arrays.keys())[0]]
    index = np.ones_like(first_array, dtype=np.bool_)
    for var_name, var_val in fixed_vars.items():
        index = index & (np.asarray(dict_of_arrays[var_name])==var_val)
    return index

def estTranProb(data, var_name, parent_names, outcomeSpace):
    """
    Calculate a dictionary probability table by ML given
    `data`, a dictionary or dataframe of observations
    `var_name`, the column of the data to be used for the conditioned variable and
    `parent_names`, a tuple of columns to be used for the parents and
    `outcomeSpace`, a dict that maps variable names to a tuple of possible outcomes
    Return a dictionary containing an estimated conditional probability table.
    """    
    var_outcomes = outcomeSpace[var_name]
    parent_outcomes = [outcomeSpace[var] for var in (parent_names)]
    # cartesian product to generate a table of all possible outcomes
    all_parent_combinations = product(*parent_outcomes)

    # Smoothing
    alpha = 1
    prob_table = odict()
    
    for i, parent_combination in enumerate(all_parent_combinations):
        parent_vars = dict(zip(parent_names, parent_combination))
        #print(parent_vars)
        parent_index = allEqualThisIndex(data, **parent_vars)
        ########we care for the previous state only, so we delete the last row#########
        parent_index=parent_index[:-1]
        #print('parent_index',len(parent_index),parent_index)
        for var_outcome in var_outcomes:
            #print('var_outcome:',var_outcome)
            var_index = (np.asarray(data[var_name])==var_outcome)
            ########we need to consider from the second state, so we delete the first row#########
            var_index=var_index[1:]
            #print('var_index',len(var_index),var_index)
            prob_table[tuple(list(parent_combination)+[var_outcome])] = ((var_index & parent_index).sum()+alpha)/(parent_index.sum() + alpha * len(var_outcomes))
            
    return {'dom': tuple(list(parent_names)+[var_name]), 'table': prob_table}


In [150]:
#Get all transition probabilities
tran_prob_table=odict()
for present,previous in previous_G.items():
    tran_prob_table[present]= estTranProb(data,present,previous,outcomeSpace)


for k in tran_prob_table.keys():
    printFactor(tran_prob_table[k])

|   r1 |   r2 |   r3 |   r1 |         Pr |
|------+------+------+------+------------|
|    0 |    0 |    0 |    0 | 0.995485   |
|    0 |    0 |    0 |    1 | 0.00451467 |
|    0 |    0 |    1 |    0 | 0.906178   |
|    0 |    0 |    1 |    1 | 0.0938215  |
|    0 |    1 |    0 |    0 | 0.858696   |
|    0 |    1 |    0 |    1 | 0.141304   |
|    0 |    1 |    1 |    0 | 0.666667   |
|    0 |    1 |    1 |    1 | 0.333333   |
|    1 |    0 |    0 |    0 | 0.0348331  |
|    1 |    0 |    0 |    1 | 0.965167   |
|    1 |    0 |    1 |    0 | 0.0953307  |
|    1 |    0 |    1 |    1 | 0.904669   |
|    1 |    1 |    0 |    0 | 0.0490196  |
|    1 |    1 |    0 |    1 | 0.95098    |
|    1 |    1 |    1 |    0 | 0.046875   |
|    1 |    1 |    1 |    1 | 0.953125   |
|   r1 |   r2 |   r4 |   r2 |         Pr |
|------+------+------+------+------------|
|    0 |    0 |    0 |    0 | 0.998584   |
|    0 |    0 |    0 |    1 | 0.00141643 |
|    0 |    0 |    1 |    0 | 0.844828   |
|    0 |   

|   r5 |   r6 |   r10 |   r11 |   r15 |   r16 |   r17 |   r18 |   r19 |   r20 |   r21 |   c3 |         Pr |
|------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------------|
|    0 |    0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |    0 | 0.896552   |
|    0 |    0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |    1 | 0.0258621  |
|    0 |    0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |    2 | 0.0172414  |
|    0 |    0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |    6 | 0.0172414  |
|    0 |    0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |    7 | 0.00862069 |
|    0 |    0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |    8 | 0.00862069 |
|    0 |    0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |    4 | 0.00862069 |
|    0 |    0 |     0 |     

In [80]:
# Office structure
office_G = {
    'r1' : ['r1','r2','r3'],
    'r2' : ['r1', 'r2','r4'],
    'r3' : ['r1', 'r3','r7'],
    'r4' : ['r2', 'r4','r8'],
    'r5' : ['r5','r6', 'r9', 'c3'],
    'r6' : ['r5','r6','c3'],
    'r7' : ['r3', 'r7','c1'],
    'r8' : ['r4', 'r8','r9'],
    'r9' : ['r5', 'r8','r9', 'r13'],
    'r10': ['r10','c3'],
    'r11': ['r11','c3'],
    'r12': ['r12','r22', 'outside'],
    'r13': ['r9', 'r13','r24'],
    'r14': ['r14','r24'],
    'r15': ['r15','c3'],
    'r16': ['r16','c3'],
    'r17': ['r17','c3'],
    'r18': ['r18','c3'],
    'r19': ['r19','c3'],
    'r20': ['r20','c3'],
    'r21': ['r21','c3'],
    'r22': ['r12', 'r22','r25'],
    'r23': ['r23','r24'],
    'r24': ['r13', 'r14', 'r23','r24'],
    'r25': ['r22', 'r25','r26'],
    'r26': ['r25', 'r26','r27'],
    'r27': ['r26', 'r27','r32'],
    'r28': ['r28','c4'],
    'r29': ['r29','r30', 'c4'],
    'r30': ['r29','r30'],
    'r31': ['r31','r32'],
    'r32': ['r27', 'r31','r32', 'r33'],
    'r33': ['r32','r33'],
    'r34': ['r34','c2'],
    'r35': ['r35','c4'],
    'c1' : ['r7', 'r25', 'c2'],
    'c2' : ['r34', 'c1', 'c4'],
    'c3' : ['r5', 'r6', 'r10', 'r11', 'r15', 'r16', 'r17', 'r18', 'r19', 'r20', 'r21', 'o1'],
    'c4' : ['r28', 'r29', 'r35', 'c2', 'o1'],
    'o1' : ['c3', 'c4'],
    'outside': ['r12']
}